## Evaluate Comp Data

In [3]:
import pandas as pd
import numpy as np

infile = "../../LapSim_Validation/2024-06-15_13_00_endurance_1.csv"

data_df = pd.read_csv(infile)

Ok - let's create a list of all the different signal names - uniquely!!

In [4]:
signals = list(data_df.loc[:, "signal"])
units = list(data_df.loc[:, "unit"])

unique_signals = []
corresponding_units = []
for i in range(0, len(signals)):
    if signals[i] not in unique_signals:
        unique_signals.append(signals[i])
        corresponding_units.append(units[i])

Now that we have a list of unique signals, we can create a dictionary of time/value pairs for each signal type

These values are listed in the "value" column of the dataframe. The time is in the "time" column.

In [10]:
# Initialize nested dict
signal_dict = {}

for label in unique_signals:
    signal_dict = {**signal_dict, label: {"Time": [], "Duration": [], "Value": [], "Unit": None}}

# Comb through dataframe and pull values and times - put them into dataframes, nested dicts, or just vectors within the dictionary
for i in range(0, len(data_df)):
    # Find the index
    thisIndex = data_df.index[i]

    # Find the signal
    thisSignal = data_df.loc[thisIndex, "signal"]

    # Find the data
    thisValue = data_df.loc[thisIndex, "value"]

    # Find the time
    thisTime = data_df.loc[thisIndex, "time"]

    # Put the data into the correct nested dict
    signal_dict[thisSignal]["Time"].append(thisTime)
    signal_dict[thisSignal]["Value"].append(thisValue)

In [11]:
# Export as pickle file
import pickle
outpath = "../../LapSim_Validation/temp_signal_dict.pkl"

with open(outpath, "wb") as file:
    pickle.dump(signal_dict, file)

Now I need to look at what measurements I actually want and how much data there is for those measurements!

Desired measurements include:

BMS:
* BMS_MaxCellTemperature
* BMS_MinCellTemperature
* BMS_ChargingCurrent
* BMS_PackVoltage
* BMS_SoC
* BMS_MinCellVoltage
* BMS_MaxCellVoltage
* BMS_TractiveSystemVoltage
* BMS_TractiveSystemCurrent
* BMS_TractiveSystemPower

INVL:
* INVL_CommandedTorque
* INVL_TorqueFeedback
* INVL_DcBusVoltage
* INVL_OutputVoltage
* INVL_DCBusCurrent
* INVL_MotorSpeed

INVR:
* INVR_CommandedTorque
* INVR_TorqueFeedback
* INVR_DcBusVoltage
* INVR_OutputVoltage
* INVR_DcBusCurrent
* INVR_MotorSpeed

Testing working with the "time" strings

In [21]:
# Reload signal dict if errors occur during debugging

import pickle
infile = "../../LapSim_Validation/temp_signal_dict.pkl"

with open(infile, "rb") as file:
    signal_dict = pickle.load(file)

Actual changes to the time format. Addition of a duration column in units of seconds.

In [22]:
import datetime

# Add a new section with "duration"?
dictkeys = list(signal_dict.keys())     # dictionary keys for iteration

# Convert "Time" to datetime format
for label in dictkeys:  # for each key
    for i in range(0, len(signal_dict[label]["Time"])): # and each time
        # Convert to datetime format
        output = signal_dict[label]["Time"][i]

        # Checking for data edge cases
        debug = output.split("-")
        if len(debug) != 4:
            print("Missing Last Strange Digit! Skip slicing step.")
        else:
            output = output[:len(output) - 6]               # cut last 6 digits cause they're weird!!

        # Check for missing milliseconds
        debug2 = output.split(".")
        if len(debug2) != 2:
            print("Missing microseconds. Different str -> time conversion.")
            output = datetime.datetime.strptime(output, '%Y-%m-%d %H:%M:%S')
        else:
            output = datetime.datetime.strptime(output, '%Y-%m-%d %H:%M:%S.%f') # convert to datetime format
        
        # Save in original location
        signal_dict[label]["Time"][i] = output

        # Add to the "duration" list
        if i == 0:
            reference_time = output.timestamp()
            signal_dict[label]["Duration"].append(0)
        else:
            signal_dict[label]["Duration"].append(output.timestamp() - reference_time)

# Check output
signal_dict[dictkeys[0]]["Duration"]

Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conversion.
Missing microseconds. Different str -> time conv

[0,
 0.09599995613098145,
 0.1959998607635498,
 0.29600000381469727,
 0.3959999084472656,
 0.495999813079834,
 0.5959999561309814,
 0.6959998607635498,
 0.7960000038146973,
 0.8959999084472656,
 1.0,
 1.0959999561309814,
 1.1959998607635498,
 1.2960000038146973,
 1.3959999084472656,
 1.495999813079834,
 1.5959999561309814,
 1.6959998607635498,
 1.7960000038146973,
 1.8959999084472656,
 2.0,
 2.0959999561309814,
 2.19599986076355,
 2.2960000038146973,
 2.3959999084472656,
 2.495999813079834,
 2.5959999561309814,
 2.69599986076355,
 2.7960000038146973,
 2.8959999084472656,
 2.998999834060669,
 3.0959999561309814,
 3.19599986076355,
 3.2960000038146973,
 3.3959999084472656,
 3.495999813079834,
 3.5959999561309814,
 3.69599986076355,
 3.7960000038146973,
 3.8959999084472656,
 3.998999834060669,
 4.0959999561309814,
 4.19599986076355,
 4.296000003814697,
 4.395999908447266,
 4.496999979019165,
 4.5959999561309814,
 4.69599986076355,
 4.796000003814697,
 4.895999908447266,
 4.998999834060669

In [6]:
bms_tsCurrent = signal_dict["BMS_TractiveSystemCurrent"]

testing = bms_tsCurrent["Time"][0]

testing = testing[:len(testing) - 6]        # remove the weird last digits

# convert from string to datetime
import datetime
dt_obj = datetime.datetime.strptime(testing,'%Y-%m-%d %H:%M:%S.%f')

reference_time = dt_obj.timestamp()         # Reference time in seconds

testing2 = bms_tsCurrent["Time"][1]          # Next point in time
testing2 = testing2[:len(testing2) - 6]
testing2 = datetime.datetime.strptime(testing2,'%Y-%m-%d %H:%M:%S.%f')

new_time = testing2.timestamp() - reference_time

new_time

# Now if I wanted to plot - maybe I would need to convert it to just solid additive time...

# CHECK OUT THIS WEBSITE TO CONVERT EVERYTHING TO A "TOTAL SECONDS FIGURE" - https://www.geeksforgeeks.org/convert-a-datetime-object-to-seconds/
# This may be helpful for my lap sim - because I do not have distance data and may need to compare patterns in time data instead...

0.09999990463256836

### Now plot some results to see how they look!

NEXT STEP:

CONVERT THE "TIME" COLUMN FROM STRINGS TO ACTUAL TIMES. AND/OR CUT DOWN THE STRINGS AT EACH MEASUREMENT (EARLIER WHEN MAKING THE DICT!)

ALSO MOVE AWAY FROM RELYING ON JUPYTER NOTEBOOK MEMORY --> SAVE THE BIG DICTIONARY TO A PICKLE FILE AND UPLOAD.

Other approaches to validation include:
1) Using the current profile and seeing if the battery voltage / power are similar
2) Using the torque request profile and seeing if the voltage/current/power are similar
3) Looking at the maximum values for current/voltage/power/torque request to determine if they are within the same range
4) Calculate energy usage from the given data and determining whether it's similar to my energy usage

--- doing the above suggestions by using my code with their data to see whether my functions and calculations are correct

In [23]:
# Export as pickle file
import pickle
outpath = "../../LapSim_Validation/signal_dict.pkl"

with open(outpath, "wb") as file:
    pickle.dump(signal_dict, file)

In [6]:
# Export as pickle file
import pickle
outpath = "../../LapSim_Validation/signal_dict.pkl"

with open(outpath, "rb") as file:
    temp_signal_dict = pickle.load(file)

In [7]:
dictkeys = list(temp_signal_dict.keys())

i = 0
for label in dictkeys:
    temp_signal_dict[label]["Unit"] = corresponding_units[i]
    i = i + 1

In [9]:
with open(outpath, "wb") as file:
    pickle.dump(temp_signal_dict, file)